# Setup dataset

In [2]:
import pandas as pd

DATA_POSITION_PATH = "data/position/"
DATA_DERIVATED_PATH = "data/derivated/"
MURIEL_POS = DATA_POSITION_PATH+"muriel/"
MARIANNE_POS = DATA_POSITION_PATH+"marianne/"
CORA_POS = DATA_POSITION_PATH+"cora/"
JUL_SEV_POS = DATA_POSITION_PATH+"07-07/"
JUL_EI_POS = DATA_POSITION_PATH+"07-08/"
JUL_THI_POS = DATA_POSITION_PATH+"07-13/"


DRIVE_POS_MAP = { 1:  (MURIEL_POS+"t_003_finalclean.tsv",(37,40.5)),        
                2:  (MURIEL_POS+"t_003_finalclean.tsv",(58,64)),        
                3:  (MURIEL_POS+"t_003_finalclean.tsv",(73,76)),
                4:  (MURIEL_POS+"t_004_finalclean.tsv",(13,16)),
                5:  (MARIANNE_POS+"t_002_finalclean.tsv",(36.3,41.5)),
                6:  (MARIANNE_POS+"t_002_finalclean.tsv",(44.4,50)),
                7:  (MARIANNE_POS+"t_003_finalclean.tsv",(3,8.9)),
                8:  (MARIANNE_POS+"t_005_finalclean.tsv",(4,9)),
                9:  (MARIANNE_POS+"t_005_finalclean.tsv",(9.2,13.4)),
                10: (MARIANNE_POS+"t_012_finalclean.tsv",(14,20.1)),
                11: (CORA_POS+"t_028_finalclean.tsv",(4.7,8.3)),
                12: (CORA_POS+"t_028_finalclean.tsv",(12.8,18.5)),
                13: (CORA_POS+"t_028_finalclean.tsv",(24.2,29.7)),
                14: (CORA_POS+"t_028_finalclean.tsv",(29.8,36.2)),
                15: (CORA_POS+"t_028_finalclean.tsv",(51.1,55)),
                16: (CORA_POS+"t_028_finalclean.tsv",(57.9,63.8)),
                17: (JUL_SEV_POS+"t_051_finalclean.tsv",(16.9,18.7)),
                18: (JUL_SEV_POS+"t_052_finalclean.tsv",(11.5,13.2)),
                19: (JUL_SEV_POS+"t_052_finalclean.tsv",(15,18)),
                20: (JUL_SEV_POS+"t_052_finalclean.tsv",(19,21.5)),
                21: (JUL_SEV_POS+"t_060_finalclean.tsv",(2.6,5)),
                22: (JUL_SEV_POS+"t_061_finalclean.tsv",(58,61)),
                23: (JUL_SEV_POS+"t_063_finalclean.tsv",(25.5,28.5)),
                24: (JUL_SEV_POS+"t_064_finalclean.tsv",(2.5,5.5)),
                25: (JUL_EI_POS+"t_052_A_finalclean.tsv",(8.1,11.8)),       
                26: (JUL_EI_POS+"t_052_B_finalclean.tsv",(11.8,15.1)),      
                27: (JUL_EI_POS+"t_052_A_finalclean.tsv",(12.15,14.7)),     
                28: (JUL_EI_POS+"t_052_B_finalclean.tsv",(8.5,11.8)),       
                29: (JUL_EI_POS+"t_053_A_finalclean.tsv",(22,24.5)), 
                30: (JUL_EI_POS+"t_053_B_finalclean.tsv",(22,24.5)),
                31: (JUL_EI_POS+"t_053_A_finalclean.tsv",(25,27.2)),       
                32: (JUL_EI_POS+"t_053_B_finalclean.tsv",(25,27.2)),
                33: (JUL_THI_POS+"t_061_A_finalclean.tsv",(29,32.1)),       
                34: (JUL_THI_POS+"t_061_B_finalclean.tsv",(29,32.1)), 
                35: (JUL_THI_POS+"t_061_A_finalclean.tsv",(74,80)),  
                36: (JUL_THI_POS+"t_061_B_finalclean.tsv",(74,80))          
              }

## Derivatives

In [3]:
import numpy as np
import pandas as pd

def group_table_by_joints(table:pd.DataFrame):
    columns = [col.replace('_X','') for col in list(table.columns)[::3]]
    result = pd.DataFrame(columns=columns)
    for j in range(0,table.shape[1],3):
        lst = []
        for i in range(table.shape[0]):
            lst.append(table.iloc[i,j:j+3].values)
        result[columns[int(j/3)]] = lst
    return result

def compute_derivatives(posTable:pd.DataFrame,timeCol,which:set={'speed'}):
    out = []
    if 'speed' in which:
        velocityTable = pd.DataFrame(np.gradient(posTable,timeCol,axis=0),columns=[f'{c}' for c in posTable.columns])
        out.append(velocityTable)
    if 'acceleration' in which:
        accelerationTable = pd.DataFrame(np.gradient(velocityTable,timeCol,axis=0),columns=[f'{c}' for c in velocityTable.columns])
        out.append(accelerationTable)
    if len(out) == 1:
        return out[0]
    return out

def compute_angular_momentum(posTables,timeCol):
    m_lfoot = 0.0145;m_rfoot = 0.0145;m_lank = 0.0465;m_rank = 0.0465
    m_lknee = 0.1;m_rknee = 0.1;m_lhip = 0.1;m_hipc = 0.139;m_rhip = 0.1
    m_spine = 0.216;m_lhand = 0.006;m_rhand = 0.006;m_lwrist = 0.016;m_rwrist = 0.016
    m_lelb = 0.028;m_relb = 0.028;m_lsho = 0.0158;m_shoc = 0.081;m_rsho = 0.0158;m_head = 0.081

    mass = [m_lfoot, m_rfoot, m_lank, m_rank, m_lknee,  m_rknee, m_lhip, m_hipc, m_rhip, m_spine, m_lhand, m_rhand, m_lwrist, m_rwrist, m_lelb, m_relb, m_lsho, m_shoc, m_rsho, m_head]
    
    posTableX, posTableY, posTableZ = posTables
    posTableX:pd.DataFrame;posTableY:pd.DataFrame;posTableZ:pd.DataFrame

    centerOfMassX = posTableX.dot(mass).div(sum(mass))
    centerOfMassY = posTableY.dot(mass).div(sum(mass))
    centerOfMassZ = posTableZ.dot(mass).div(sum(mass))

    radiusQX = posTableX.sub(centerOfMassX,axis=0)
    radiusQY = posTableY.sub(centerOfMassY,axis=0)
    radiusQZ = posTableZ.sub(centerOfMassZ,axis=0)

    def merge_tables(xTable,yTable,zTable):
        mergedTable = pd.DataFrame()
        for j in range(xTable.shape[1]):
            mergedTable = pd.concat([mergedTable,xTable.iloc[:,j],yTable.iloc[:,j],zTable.iloc[:,j]],axis=1)
        return mergedTable
    
    radiusQTable = merge_tables(radiusQX,radiusQY,radiusQZ)
    p = compute_derivatives(merge_tables(posTableX,posTableY,posTableZ),timeCol).mul(np.repeat(mass,3))

    def split_table(table,into="xyz"):
        if into == "xyz":
            return table.iloc[:,::3],table.iloc[:,1::3],table.iloc[:,2::3]
        elif into == "points":
            return [table.iloc[:,j:j+3] for j in range(0,table.shape[1],3)]
        
    p = [point.to_numpy() for point in split_table(p,into="points")]
    r = [point.to_numpy() for point in split_table(radiusQTable, into="points")]
    l = np.cross(p,r)

    angularMomentumTable = radiusQTable*0
    for j,point in enumerate(l):
        for i in range(len(point)):
            angularMomentumTable.iloc[i,3*j:3*j+3] = point[i]
    return angularMomentumTable

## Similarity functions 

In [4]:
from scipy.spatial.distance import cosine
def compute_cosine_similarity_on_joints(featureTable,normalized:None or int=None):
    jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
    jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]

    jointsFrom = list(map(lambda x: x-1,jointsFrom))
    jointsTo = list(map(lambda x: x-1,jointsTo))

    cosineSim = pd.DataFrame()
    columnNames = list(featureTable.columns)
    for jointFrom,jointTo in zip(jointsFrom,jointsTo):
        cosineSim[columnNames[jointFrom]+" - "+columnNames[jointTo]] = featureTable.apply(lambda row: 2 - cosine(row[columnNames[jointFrom]],row[columnNames[jointTo]]),axis=1)
    return cosineSim if normalized is None else cosineSim.div(cosineSim.max(axis=1))*normalized

from numpy.linalg import norm
def compute_inverse_difference_on_joints(featureTable):
    jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
    jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
    jointsFrom = list(map(lambda x: x-1,jointsFrom))
    jointsTo = list(map(lambda x: x-1,jointsTo))

    invDiffModulus = pd.DataFrame()
    eps = 10**(-9)  # To avoid division by 0
    columnNames = list(featureTable.columns)
    for jointFrom,jointTo in zip(jointsFrom,jointsTo):
        invDiffModulus[columnNames[jointFrom]+" - "+columnNames[jointTo]] = featureTable.apply(lambda row: norm(row[columnNames[jointFrom]]-row[columnNames[jointTo]])+eps,axis=1)
    return 1 / invDiffModulus
    

def group_table_by_joints(table):
    columns = [col.replace('_X','') for col in list(table.columns)[::3]]
    result = pd.DataFrame(columns=columns)
    for j in range(0,table.shape[1],3):
        lst = []
        for i in range(table.shape[0]):
            lst.append(table.iloc[i,j:j+3].values)
        result[columns[int(j/3)]] = lst
    return result

## Optional functions for rotating, filtering, plotting

In [5]:
def filter_and_rotate_table(picked:int,posTable:pd.DataFrame):
    if 1 <= picked <= 10:
        posTableY = posTable.iloc[:,::3]
        posTableZ = posTable.iloc[:,1::3]
        posTableX = posTable.iloc[:,2::3]
    elif 11 <= picked <= 16:
        posTableX = posTable.iloc[:,::3]
        posTableZ = posTable.iloc[:,1::3]
        posTableY = posTable.iloc[:,2::3]
    elif (17 <= picked <= 20 and picked != 19) or picked >= 25:
        posTableY = posTable.iloc[:,::3]
        posTableX = posTable.iloc[:,1::3]
        posTableZ = posTable.iloc[:,2::3]
    else:
        posTableX = posTable.iloc[:,::3]
        posTableY = posTable.iloc[:,1::3]
        posTableZ = posTable.iloc[:,2::3]

    posTableX = posTableX.rename(columns=dict(zip(list(posTableX),[name.replace('Z','X').replace('Y','X') for name in list(posTableX)])))
    posTableY = posTableY.rename(columns=dict(zip(list(posTableY),[name.replace('Z','Y').replace('X','Y') for name in list(posTableY)])))
    posTableZ = posTableZ.rename(columns=dict(zip(list(posTableZ),[name.replace('X','Z').replace('Y','Z') for name in list(posTableZ)])))

    return (posTableX,posTableY,posTableZ)


from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
import numpy as np

def draw_skeleton(posTable:tuple,picked:int):
    posTableX,posTableY,posTableZ = posTable
    posTableX:pd.DataFrame
    posTableY:pd.DataFrame
    posTableZ:pd.DataFrame
    plt.close("all")
    %matplotlib
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')


    minMax = np.zeros((2,3))
    minMax[0,:] = [posTableX.values.min(),posTableY.values.min(),posTableZ.values.min()]
    minMax[1,:] = [posTableX.values.max(),posTableY.values.max(),posTableZ.values.max()]

    print(minMax)

    # Set appropriate axis limits
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    # Set the window title
    fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

    # Set the initial time index
    time_index = 0

    # Function to update the plot based on the slider value
    def update_plot(val):
        ax.cla()  # Clear the previous plot

        # Filter the data based on the current time index
        filteredX = posTableX.iloc[val]
        filteredY = posTableY.iloc[val]
        filteredZ = posTableZ.iloc[val]

        ax.scatter(filteredX,filteredY,filteredZ)

        ax.set_xlim([minMax[0,0],minMax[1,0]])
        ax.set_ylim([minMax[0,1],minMax[1,1]])
        ax.set_zlim([minMax[0,2],minMax[1,2]])

        ax.set_xlabel('X', fontsize=12)
        ax.set_ylabel('Y', fontsize=12)
        ax.set_zlabel('Z', fontsize=12)
        ax.set_title("Movement "+str(picked))

        fig.canvas.draw_idle()

    # Create a slider widget
    slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
    maxValue = posTable.shape[0]-1
    slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


    # Define a function to update the slider value with the mouse wheel
    def on_scroll(event):
        if event.button == 'down':
            if slider.val + slider.valstep*5 <= maxValue:
                slider.set_val(slider.val + slider.valstep*2)
        elif event.button == 'up':
            if slider.val - slider.valstep*10 >= 0:
                slider.set_val(slider.val - slider.valstep*2)


    # Connect the mouse wheel event to the function
    fig.canvas.mpl_connect('scroll_event', on_scroll)


    # Register the update_plot function with the slider widget
    slider.on_changed(update_plot)

    # Initial plot
    update_plot(time_index)

    # Show the plot
    plt.show()

# Filling up the dataset

Infos about the dataset
```
Video Names Mapping and manual annotation:

Muriel t_003.1 ==> 1 ==> Shoc/Rsho
Muriel t_003.2 ==> 2 ==> Hipc/Head !!
Muriel t_003.3 ==> 3 ==> Rhip/Hipc
Muriel t_004 ==> 4 ==> Shoc/Lsho

Marianne t_002.1 ==> 5 ==> Shoc/Head
Marianne t_002.2 ==> 6 ==> Rknee/Rank
Marianne t_003 ==> 7 ==> Lsho/Shoc
Marianne t_005.1 ==> 8 ==> Rsho/Shoc
Marianne t_005.2 ==> 9 ==> Rank/Hipc !!
Marianne t_012 ==> 10 ==> Shoc/Lsho !!

Cora t_028.1 ==> 11 ==> Rwrist/Rhand
Cora t_028.2 ==> 12 ==> Relb/Head
Cora t_028.3 ==> 13 ==> Lank/Lhip
Cora t_028.4 ==> 14 ==> Lknee/Lhip
Cora t_028.5 ==> 15 ==> Lwrist/Lsho
Cora t_028.6 ==> 16 ==> Hipc/Shoc !!

No OoM.1 (07-07 t_052) ==> 19 ==> No
No OoM.2 (07-07 t_052) ==> 18 ==> No

07-07 t_051 ==> 17 ==> Rknee/Lknee !!
07-07 t_052 ==> 20 ==> Shoc/Hipc !!
07-07 t_060 ==> 21 ==> Rsho/Relb
07-07 t_061 ==> 22 ==> Lelb/Shoc
07-07 t_063 ==> 23 ==> Lelb/Shoc
07-07 t_064 ==> 24 ==> Relb/Rwrist

07-08 t_052_A.1 ==> 25 ==> Lknee/Rknee !!
07-08 t_052_B.1 ==> 26 ==> Lknee/Shoc
07-08 t_052_A.2 ==> 27 ==> Shoc/Lelb
07-08 t_052_B.2 ==> 28 ==> Rknee/Lknee !!
07-08 t_053_A.1 ==> 29 ==> Lknee/Rknee
07-08 t_053_B.1 ==> 30 ==> Rknee/Shoc
07-08 t_053_A.2 ==> 31 ==> Rhip/Shoc
07-08 t_053_B.2 ==> 32 ==> Rknee/Relb

07-13 t_061_A.1 ==> 33 ==> Lelb/Relb
07-13 t_061_B.1 ==> 34 ==> Lknee/Rknee
07-13 t_061_A.2 ==> 35 ==> Rknee/Shoc
07-13 t_061_B.2 ==> 36 ==> Rsho/Relb
```

In [2]:
edgeNames = ['Lfoot-Lankle',           
          'Rfoot-Rankle',           
          'Lankle-Lknee',           
          'Rankle-Rknee',           
          'Lknee-Lhip',
          'Rknee-Rhip',
          'Lhip-Chip',
          'Rhip-Chip',
          'Chip-spine',
          'spine-Cshoulder',
          'Lhand-Lwrist',
          'Rhand-Rwrist',
          'Lwrist-Lelbow',
          'Rwrist-Relbow',
          'Lelbow-Lshoulder',
          'Relbow-Rshoulder',       
          'Lshoulder-Cshoulder',    
          'Rshoulder-Cshoulder',    
          'Cshoulder-head']         
jointNames = [  'left_foot',
                'right_foot',
                'left_ankle',
                'right_ankle',
                'left_knee',
                'right_knee',
                'left_hip',
                'hip_center',
                'right_hip', 
                'spine',
                'left_hand',
                'right_hand',
                'left_wrist',
                'right_wrist',
                'left_elbow',
                'right_elbow',
                'left_shoulder',
                'shoulder_center',
                'right_shoulder',
                'head'  ]
y_mask =     {1:    [edgeNames.index('Rshoulder-Cshoulder')],
              2:    [edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder'),edgeNames.index('Cshoulder-head')],
              3:    [edgeNames.index('Rhip-Chip')],
              4:    [edgeNames.index('Lshoulder-Cshoulder')],
              5:    [edgeNames.index('Cshoulder-head')],
              6:    [edgeNames.index('Rankle-Rknee')],
              7:    [edgeNames.index('Lshoulder-Cshoulder')],
              8:    [edgeNames.index('Rshoulder-Cshoulder')],
              9:    [edgeNames.index('Rankle-Rknee'),edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip')],
              10:   [edgeNames.index('Lshoulder-Cshoulder')],
              11:   [edgeNames.index('Rhand-Rwrist')],
              12:   [edgeNames.index('Relbow-Rshoulder'),edgeNames.index('Rshoulder-Cshoulder'),edgeNames.index('Cshoulder-head')],
              13:   [edgeNames.index('Lankle-Lknee'),edgeNames.index('Lknee-Lhip')],
              14:   [edgeNames.index('Lknee-Lhip')],
              15:   [edgeNames.index('Lwrist-Lelbow'),edgeNames.index('Lelbow-Lshoulder')],
              16:   [edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              17:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Lknee-Lhip')],
              18:   [],
              19:   [],
              20:   [edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              21:   [edgeNames.index('Relbow-Rshoulder')],
              22:   [edgeNames.index('Lelbow-Lshoulder'),edgeNames.index('Lshoulder-Cshoulder')],
              23:   [edgeNames.index('Lelbow-Lshoulder'),edgeNames.index('Lshoulder-Cshoulder')],
              24:   [edgeNames.index('Rwrist-Relbow')],
              25:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              26:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              27:   [edgeNames.index('Lshoulder-Cshoulder'),edgeNames.index('Lelbow-Lshoulder')],
              28:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              29:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              30:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              31:   [edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder')],
              32:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder'),edgeNames.index('Rshoulder-Cshoulder'),edgeNames.index('Relbow-Rshoulder')],
              33:   [edgeNames.index('Lelbow-Lshoulder'),edgeNames.index('Lshoulder-Cshoulder'),edgeNames.index('Rshoulder-Cshoulder'),edgeNames.index('Relbow-Rshoulder')],
              34:   [edgeNames.index('Lknee-Lhip'),edgeNames.index('Lhip-Chip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Rknee-Rhip')],
              35:   [edgeNames.index('Rknee-Rhip'),edgeNames.index('Rhip-Chip'),edgeNames.index('Chip-spine'),edgeNames.index('spine-Cshoulder'),edgeNames.index('Rshoulder-Cshoulder')],
              36:   [edgeNames.index('Relbow-Rshoulder')]
              }
for edgeIndex in y_mask[35]:
    print(edgeNames[edgeIndex])

Rknee-Rhip
Rhip-Chip
Chip-spine
spine-Cshoulder
Rshoulder-Cshoulder


In [42]:
import pandas as pd
from numpy.linalg import norm

columns = [elem for tup in [('X_'+pair[0]+'_avg','X_'+pair[1]+'_var') for pair in zip(jointNames,jointNames)] for elem in tup] + ['Y_OneHot_'+edgeName for edgeName in edgeNames]
column_types = [float if j < len(columns)-len(edgeNames) else bool for j,k in enumerate(columns)]
dataset = pd.DataFrame(index=np.arange(len(DRIVE_POS_MAP.keys())), columns=columns).astype(dict(zip(columns,column_types)))

# Helper function to calculate mean and variance of every joint
calc_mean_and_var = lambda j: (
    speedTable.iloc[:, j : j + 3].apply(lambda row: norm(row), axis=1).mean(),
    speedTable.iloc[:, j : j + 3].apply(lambda row: norm(row), axis=1).var(),
)
oneHotEncoding = np.zeros(len(edgeNames),dtype=bool)

for i,indx_in_drive in enumerate(DRIVE_POS_MAP.keys()):
    data = DRIVE_POS_MAP[indx_in_drive]
    table = pd.read_table(data[0])
    startEndSeconds = data[1]
    timeCol = table[table["Time"].between(startEndSeconds[0],startEndSeconds[1])]["Time"]
    table = table[table["Time"].between(startEndSeconds[0],startEndSeconds[1])].iloc[:,1:]
    speedTable = compute_derivatives(table,timeCol) 

    # Assign the calculated values to the dataset
    for j in range(0,len(speedTable.columns),3):
        dataset.loc[i, f'X_{jointNames[j//3]}_avg'], dataset.loc[i, f'X_{jointNames[j//3]}_var'] = calc_mean_and_var(j)

    oneHotEncoding[y_mask[indx_in_drive]] = True
    dataset.iloc[i,-len(edgeNames):] = oneHotEncoding
    oneHotEncoding[y_mask[indx_in_drive]] = False

dataset

,X_left_foot_avg,X_left_foot_var,X_right_foot_avg,X_right_foot_var,X_left_ankle_avg,X_left_ankle_var,X_right_ankle_avg,X_right_ankle_var,X_left_knee_avg,X_left_knee_var,...,Y_OneHot_spine-Cshoulder,Y_OneHot_Lhand-Lwrist,Y_OneHot_Rhand-Rwrist,Y_OneHot_Lwrist-Lelbow,Y_OneHot_Rwrist-Relbow,Y_OneHot_Lelbow-Lshoulder,Y_OneHot_Relbow-Rshoulder,Y_OneHot_Lshoulder-Cshoulder,Y_OneHot_Rshoulder-Cshoulder,Y_OneHot_Cshoulder-head
0,0.007774,7.058829e-05,0.016240,2.744935e-04,0.027433,0.000534,0.052894,1.818841e-03,0.089811,0.007160,...,False,False,False,False,False,False,False,False,True,False
1,0.011890,1.629459e-04,0.020960,6.396525e-04,0.036153,0.001080,0.067759,4.399652e-03,0.144923,0.009984,...,True,False,False,False,False,False,False,False,False,True
2,0.196221,9.350937e-02,1.767068,3.546706e+00,0.124725,0.018970,2.213771,3.007082e+00,0.258535,0.042587,...,False,False,False,False,False,False,False,False,False,False
3,0.153154,3.788091e-02,0.572844,6.323509e-01,0.168056,0.048041,0.495058,3.406924e-01,0.278883,0.055658,...,False,False,False,False,False,False,False,True,False,False
4,0.003769,2.251460e-05,0.005164,3.626099e-05,0.007432,0.000034,0.009989,1.127561e-04,0.038700,0.000678,...,False,False,False,False,False,False,False,False,False,True
5,0.007820,6.452291e-05,0.389714,1.181945e-01,0.012321,0.000108,0.422730,7.520250e-02,0.096670,0.004867,...,False,False,False,False,False,False,False,False,False,False
6,0.000617,2.500998e-07,0.000552,1.632158e-07,0.001986,0.000020,0.001373,8.440537e-07,0.007852,0.000026,...,False,False,False,False,False,False,False,True,False,False
7,0.001831,2.373220e-06,0.003056,1.134762e-05,0.161436,0.544280,0.004217,3.888733e-05,0.019690,0.000283,...,False,False,False,False,False,False,False,False,True,False
8,0.007563,4.565466e-05,0.295923,5.387698e-01,0.199153,0.630415,0.338642,4.455530e-01,0.093191,0.005855,...,False,False,False,False,False,False,False,False,False,False
9,0.029616,8.773440e-04,0.016128,3.522086e-04,0.057657,0.001865,0.034971,1.311301e-03,0.127043,0.004649,...,False,False,False,False,False,False,False,True,False,False
